In [1]:
import sys, os, importlib
import rasterio

import numpy as np
import pandas as pd
import geopandas as gpd
import osmnx as ox
import GOSTnets as gn
import skimage.graph as graph

from rasterio.mask import mask
from rasterio import features
from shapely.geometry import box, Point, Polygon
from scipy.ndimage import generic_filter
from pandana.loaders import osm

sys.path.append("../..")

import infrasap.market_access as ma
import infrasap.rasterMisc as rMisc

In [2]:
global_friction_file    = "/home/public/Data/GLOBAL/INFRA/FRICTION_2015/2015_friction_surface_v1.geotiff"
global_friction_noOcean = "/home/public/Data/GLOBAL/INFRA/FRICTION_2015/2015_friction_surface_v1_no_ocean_travel.tif"
ports_file = "/home/wb411133/data/Global/INFRA/PORTS/major_ports.shp"



In [3]:
if not os.path.exists(global_friction_noOcean):# Burn the oceans in as a no travel zone
    inG = rasterio.open(global_friction_file)
    inG_data = inG.read()[0,:,:]
    inG_data = inG_data * 1000
    
    ocean_file = "/home/public/Data/GLOBAL/ADMIN/OCEAN/ne_10m_ocean.shp"
    cMeta = inG.meta.copy()

    inO = gpd.read_file(ocean_file)
    shapes = ((row.geometry,999999) for idx, row in inO.iterrows())
    burned = features.rasterize(shapes=shapes, out_shape=(cMeta['height'], cMeta['width']), 
                                transform=cMeta['transform'], dtype=cMeta['dtype'])
    inG_combo = inG_data + burned
    out_file = "/home/wb411133/temp/2015_friction_surface_v1_no_ocean_travel.tif"
    with rasterio.open(out_file, 'w', **cMeta) as out:
        out.write_band(1, inG_combo)
   

In [4]:
inG = rasterio.open(global_friction_noOcean)
inP = gpd.read_file(ports_file)

inG_data = inG.read()[0,:,:]
mcp = graph.MCP_Geometric(inG_data)

In [6]:
inP = gpd.read_file(ports_file)

# Optional processing of ports

In [ ]:
# Attach ISO3 codes and regions to ports
global_boundaries =  "/home/public/Data/GLOBAL/ADMIN/Admin0_Polys.shp"
inB = gpd.read_file(global_boundaries)

In [ ]:
inP = gpd.read_file(ports_file)
if inP.crs != inB.crs:
    inP = inP.to_crs(inB.crs)
    
inP['ISO3'] = ''
inP['REGION'] = ''
for idx, row in inP.iterrows():
    sel_country = inB.loc[inB.intersects(row['geometry'])]
    if sel_country.shape[0] > 0:
        inP.loc[idx,'ISO3'] = sel_country['ISO3'].iloc[0]
        inP.loc[idx,'REGION'] = sel_country['Region'].iloc[0]
    else:
        print(f"{idx}: {row['Postal']}")
              
inP = inP.to_crs({'init':'epsg:4326'})
inP.to_file(ports_file)

In [16]:
# Determine which ports are accidentally in the ocean
ocean_layer = "/home/public/Data/GLOBAL/ADMIN/OCEAN/ne_10m_ocean.shp"
inO = gpd.read_file(ocean_layer)
inO.head()

,featurecla,scalerank,min_zoom,geometry
0,Ocean,0,0,(POLYGON ((59.91602623800009 -67.4004859349999...


In [22]:
inP = gpd.read_file(ports_file)

In [23]:
inP_idx = inP['geometry'].apply(lambda x: x.intersects(inO['geometry'].iloc[0]))

In [24]:
inP.loc[inP_idx].to_file(ports_file.replace(".shp", "_BAD.shp"))

In [35]:
bad_p = gpd.read_file(ports_file.replace(".shp", "_BAD.shp"))
all_files = os.listdir(out_folder)
for idx, row in bad_p.iterrows():
    name = row['OBJECTID']    
    for inF in all_files:
        if str(name) == inF.split("_")[-1].split(".")[0]:
            os.remove(os.path.join(out_folder, inF))

In [40]:
bad_p

,OBJECTID,Postal,Lat,Lng,Type,ISO3,REGION,geometry
0,56,"Owendo,Gabon",0.29123,9.50465,General,GAB,Sub-Saharan Africa,POINT (9.504894134324367 0.2886314659927493)
1,137,"Illigan,Philippines",8.22890,124.24344,Bulk,PHL,East Asia & Pacific,POINT (124.24344 8.228899999999998)
2,129,"Cagayan De Oro,Philippines",8.48222,124.64722,General,PHL,East Asia & Pacific,POINT (124.64722 8.482220000000003)
3,134,"Surigao,Philippines",9.78900,125.49500,Bulk,PHL,East Asia & Pacific,POINT (125.495 9.788999999999993)


# Run Market Access

In [37]:
#selP = inP.loc[inP['REGION'] == 'South Asia']
inP = gpd.read_file(ports_file)

In [41]:
travel_time_thresholds = [((24*60) * x) for x in [0.25,0.5,1,2]]
out_folder = os.path.join(os.path.dirname(ports_file), "TRAVEL_TIMES")
if not os.path.exists(out_folder):
    os.makedirs(out_folder)
port_files = []
for idx, row in inP.iterrows():
    name = row['OBJECTID']    
    out_file = os.path.join(out_folder, "PORT_ISOCHRONES_NO_OCEAN_%s.shp" % name)
    port_files.append(out_file)
    if not os.path.exists(out_file):
        print("Processing %s" % name)
        current_p = pd.DataFrame(inP.loc[idx,]).transpose()
        travel_times = ma.generate_feature_vectors(inG, mcp, current_p, travel_time_thresholds)
        travel_times.to_file(out_file)
    else:
        print("%s already processed" % name)
        
    

Processing 12
11:31:42	1 of 1
Processing 19
11:44:54	1 of 1
Processing 20
11:54:55	1 of 1
Processing 23
12:04:50	1 of 1
Processing 24
12:15:00	1 of 1
Processing 41
12:25:19	1 of 1
Processing 46
12:35:12	1 of 1
Processing 47
12:44:57	1 of 1
Processing 51
12:54:45	1 of 1
Processing 63
13:04:21	1 of 1
Processing 74
13:14:16	1 of 1
Processing 80
13:25:39	1 of 1
Processing 82
13:36:38	1 of 1
Processing 89
13:46:22	1 of 1
Processing 117
13:56:06	1 of 1
Processing 122
14:08:12	1 of 1
Processing 124
14:17:44	1 of 1
Processing 132
14:29:06	1 of 1
Processing 138
14:38:44	1 of 1
Processing 152
14:48:53	1 of 1
Processing 165
14:58:41	1 of 1
Processing 166
15:09:53	1 of 1
Processing 173
15:20:57	1 of 1
Processing 175
15:31:49	1 of 1
Processing 103
15:42:36	1 of 1
Processing 102
15:52:52	1 of 1
Processing 104
16:03:02	1 of 1
Processing 105
16:13:30	1 of 1
Processing 55
16:23:48	1 of 1
Processing 62
16:33:39	1 of 1
Processing 61
16:43:31	1 of 1
Processing 67
16:53:20	1 of 1
Processing 114
17:02:59	1 

In [42]:
port_files

['/home/wb411133/data/Global/INFRA/PORTS/TRAVEL_TIMES/PORT_ISOCHRONES_NO_OCEAN_12.shp',
 '/home/wb411133/data/Global/INFRA/PORTS/TRAVEL_TIMES/PORT_ISOCHRONES_NO_OCEAN_19.shp',
 '/home/wb411133/data/Global/INFRA/PORTS/TRAVEL_TIMES/PORT_ISOCHRONES_NO_OCEAN_20.shp',
 '/home/wb411133/data/Global/INFRA/PORTS/TRAVEL_TIMES/PORT_ISOCHRONES_NO_OCEAN_23.shp',
 '/home/wb411133/data/Global/INFRA/PORTS/TRAVEL_TIMES/PORT_ISOCHRONES_NO_OCEAN_24.shp',
 '/home/wb411133/data/Global/INFRA/PORTS/TRAVEL_TIMES/PORT_ISOCHRONES_NO_OCEAN_41.shp',
 '/home/wb411133/data/Global/INFRA/PORTS/TRAVEL_TIMES/PORT_ISOCHRONES_NO_OCEAN_46.shp',
 '/home/wb411133/data/Global/INFRA/PORTS/TRAVEL_TIMES/PORT_ISOCHRONES_NO_OCEAN_47.shp',
 '/home/wb411133/data/Global/INFRA/PORTS/TRAVEL_TIMES/PORT_ISOCHRONES_NO_OCEAN_51.shp',
 '/home/wb411133/data/Global/INFRA/PORTS/TRAVEL_TIMES/PORT_ISOCHRONES_NO_OCEAN_63.shp',
 '/home/wb411133/data/Global/INFRA/PORTS/TRAVEL_TIMES/PORT_ISOCHRONES_NO_OCEAN_74.shp',
 '/home/wb411133/data/Global/INF

In [43]:
try:
    del final
except:
    pass
for p in port_files:
    current_p = gpd.read_file(p)
    current_p['PORT'] = p.split("_")[-1].replace(".shp","")
    try:
        final = final.append(current_p)
    except:
        final = current_p

In [44]:
final.to_file(os.path.join(out_folder, "COMBO_all_ports.shp"))

In [45]:
out_folder

'/home/wb411133/data/Global/INFRA/PORTS/TRAVEL_TIMES'

# Zonal Stats

In [ ]:
inShape = os.path.join(out_folder, "COMBO_all_ports.shp")